In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/CS7643/Project/data/tokenizer'

/content/drive/MyDrive/CS7643/Project/data/tokenizer


Ref: https://towardsdatascience.com/how-to-build-a-wordpiece-tokenizer-for-bert-f505d97dddbb

In [ ]:
!pip install datasets
!pip install transformers
!pip install tokenizers

In [68]:
from datasets import load_dataset

Generate Tokenizer from News dataset

In [ ]:
dataset = load_dataset('ag_news')

Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [ ]:
import os
os.mkdir('./ag-news')


In [ ]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n', '\s')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'./ag-news/file_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we may have leftover samples, we save those now too
with open(f'./ag-news/file_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/120000 [00:00<?, ?it/s]

In [ ]:
from pathlib import Path
paths = [str(x) for x in Path('./ag-news').glob('**/*.txt')]
paths[:5]

['ag-news/file_0.txt',
 'ag-news/file_1.txt',
 'ag-news/file_2.txt',
 'ag-news/file_3.txt',
 'ag-news/file_4.txt']

In [ ]:
len(paths)

25

In [ ]:
# !pip install tokenizers
from tokenizers import BertWordPieceTokenizer

# initialize
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)
# and train
tokenizer.train(files=paths, vocab_size=30_000, min_frequency=2,
                limit_alphabet=1000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

In [ ]:
os.mkdir('./news_tokenizer')
tokenizer.save_model('./news_tokenizer')

['./news_tokenizer/vocab.txt']

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./news_tokenizer')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generate Tokenizer from our dataset

In [70]:
%cd '/content/drive/MyDrive/CS7643/Project'

/content/drive/MyDrive/CS7643/Project


In [71]:
# All data will be loaded together unless data_files is specified
dataset = load_dataset('data', data_files ={'train':'train.csv','test':'test.csv'})
dataset = dataset.rename_column("Unnamed: 0", 'id')

Using custom data configuration data-cf9ebd5eebbe0a50


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/data-cf9ebd5eebbe0a50/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [73]:
dataset['train']

Dataset({
    features: ['id', 'text', 'label'],
    num_rows: 35918
})

In [75]:
%cd '/content/drive/MyDrive/CS7643/Project/data/tokenizer'

/content/drive/MyDrive/CS7643/Project/data/tokenizer


In [76]:
import os
os.mkdir('./fake-news')


In [77]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n', '\s')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'./fake-news/file_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we may have leftover samples, we save those now too
with open(f'./fake-news/file_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/35918 [00:00<?, ?it/s]

In [79]:
from pathlib import Path
paths = [str(x) for x in Path('./fake-news').glob('**/*.txt')]
paths[:5]

['fake-news/file_0.txt',
 'fake-news/file_1.txt',
 'fake-news/file_2.txt',
 'fake-news/file_3.txt',
 'fake-news/file_4.txt']

In [85]:
# !pip install tokenizers
from tokenizers import BertWordPieceTokenizer

# initialize
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)
# and train
tokenizer.train(files=paths, vocab_size=30_000, min_frequency=2,
                limit_alphabet=1000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

In [86]:
os.mkdir('./fake_tokenizer')
tokenizer.save_model('./fake_tokenizer')

['./fake_tokenizer/vocab.txt']

In [87]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./fake_tokenizer')